In [1]:
import numpy as np
import pickle
import operator
import pandas as pd
import jieba
from language.langconv import *

In [2]:
def Traditional2Simplified(sentence):
    sentence = Converter('zh-hans').convert(sentence)
    return sentence
with open('cmn.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n');
eng=[]
cns=[]
print(lines[:10])
for pos, line in enumerate(lines):
    line = line.split('\t')
    e = line[0][:-1] + " " + line[0][-1:]
    c = line[1]
    eng.append(' '.join(e.split(' ')))
    cns.append(' '.join(jieba.lcut(Traditional2Simplified(c).strip(), cut_all=False)))
print(len(cns))
print(eng[10:20])
print(cns[10:20])

maxLen=18
for pos, seq in enumerate(eng):
    seq_list = seq.split(' ')
    if len(seq_list) > maxLen:
        seq_list = seq_list[:maxLen]
    eng[pos] = ' '.join(seq_list)
for pos, seq in enumerate(cns):
    seq_list = seq.split(' ')    
    if len(seq_list) > maxLen:
        seq_list = seq_list[:maxLen]
    cns[pos] = ' '.join(seq_list)

cns_a = ['BOS '+i+' EOS' for i in cns]
cns_b = [i+' EOS' for i in cns]
print(cns_a[10:20])
print(cns_b[10:20])

Building prefix dict from the default dictionary ...
Loading model from cache E:\Temp\jieba.cache


['Hi.\t嗨。', 'Hi.\t你好。', 'Run.\t你用跑的。', 'Wait!\t等等！', 'Hello!\t你好。', 'I try.\t让我来。', 'I won!\t我赢了。', 'Oh no!\t不会吧。', 'Cheers!\t乾杯!', 'He ran.\t他跑了。']


Loading model cost 0.781 seconds.
Prefix dict has been built succesfully.


20403
['Hop in .', 'I lost .', 'I quit .', "I'm OK .", 'Listen .', 'No way !', 'No way !', 'Really ?', 'Try it .', 'We try .']
['跳进来 。', '我 迷失 了 。', '我 退出 。', '我 没事 。', '听 着 。', '不 可能 ！', '没门 ！', '你 确定 ？', '试试 吧 。', '我们 来 试试 。']
['BOS 跳进来 。 EOS', 'BOS 我 迷失 了 。 EOS', 'BOS 我 退出 。 EOS', 'BOS 我 没事 。 EOS', 'BOS 听 着 。 EOS', 'BOS 不 可能 ！ EOS', 'BOS 没门 ！ EOS', 'BOS 你 确定 ？ EOS', 'BOS 试试 吧 。 EOS', 'BOS 我们 来 试试 。 EOS']
['跳进来 。 EOS', '我 迷失 了 。 EOS', '我 退出 。 EOS', '我 没事 。 EOS', '听 着 。 EOS', '不 可能 ！ EOS', '没门 ！ EOS', '你 确定 ？ EOS', '试试 吧 。 EOS', '我们 来 试试 。 EOS']


In [3]:
import  pickle
counts = {}
BE = ['BOS', 'EOS']
for word_list in eng+cns + BE:
    for word in word_list.split(' '):
        counts[word] = counts.get(word, 0) + 1 
word_to_index = {}
for pos, i in enumerate(counts.keys()):
    word_to_index[i] = pos
    
index_to_word = {}
for pos, i in enumerate(counts.keys()):
    index_to_word[pos] = i
    
vocab_bag =list(word_to_index.keys())
with open('word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)
with open('vocab_bag.pkl', 'wb') as f:
    pickle.dump(vocab_bag, f, pickle.HIGHEST_PROTOCOL)

print('vocab_bag: ', len(vocab_bag))


vocab_bag:  17751


In [4]:
eng = np.array([[word_to_index[w] for w in i.split(' ')] for i in eng])
cns_a = np.array([[word_to_index[w] for w in i.split(' ')] for i in cns_a])
cns_b = np.array([[word_to_index[w] for w in i.split(' ')] for i in cns_b])
print(eng[:3])
print(cns_a[:3])
print(cns_b[:3])

[list([0, 1]) list([0, 1]) list([2, 1])]
[list([17749, 7300, 7301, 17750]) list([17749, 7302, 7301, 17750])
 list([17749, 7303, 7304, 7305, 7306, 7301, 17750])]
[list([7300, 7301, 17750]) list([7302, 7301, 17750])
 list([7303, 7304, 7305, 7306, 7301, 17750])]


In [5]:
import os 
import numpy as np
print('eng: ', len(eng), '\n', 'cns: ', len(cns))
np.save('eng.npy', eng[:20000])
np.save('cns_a.npy', cns_a[:20000])
np.save('cns_b.npy', cns_b[:20000])
print('Done!')

eng:  20403 
 cns:  20403
Done!


In [6]:
import numpy as np
import pickle
import operator
eng = np.load('eng.npy')
cns_a = np.load('cns_a.npy')
cns_b = np.load('cns_b.npy')
print('cns_a.shape: ', cns_a.shape)
with open('word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
for i, j in word_to_index.items():
    word_to_index[i] = j + 1
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key
pad_eng = eng
pad_cns_a = cns_a
pad_cns_b = cns_b
maxLen=20
for pos, i in enumerate(pad_eng):
    for pos_, j in enumerate(i):
        i[pos_] = j + 1
    if(len(i) > maxLen):
        pad_eng[pos] = i[:maxLen]
    
for pos, i in enumerate(pad_cns_a):
    for pos_, j in enumerate(i):
        i[pos_] = j + 1
    if(len(i) > maxLen):
        pad_cns_a[pos] = i[:maxLen]
for pos, i in enumerate(pad_cns_b):
    for pos_, j in enumerate(i):
        i[pos_] = j + 1
    if(len(i) > maxLen):
        pad_cns_b[pos] = i[:maxLen]
np.save('cns_o.npy', pad_cns_b)        
    
with open('vocab_bag.pkl', 'rb') as f:
    words = pickle.load(f)
vocab_size = len(word_to_index) + 1
print('word_to_vec_map: ', len(list(words)))
print('vocab_size: ', vocab_size)

cns_a.shape:  (20000,)
word_to_vec_map:  17751
vocab_size:  17752


In [7]:
from keras.preprocessing import sequence
#后端padding
pad_eng = sequence.pad_sequences(pad_eng, maxlen=maxLen,
                                      dtype='int32', padding='post', 
                                       truncating='post')
pad_cns = sequence.pad_sequences(pad_cns_a, maxlen=maxLen,
                                 dtype='int32', padding='post',
                                 truncating='post')

def get_file_list(file_path):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        dir_list = sorted(dir_list, key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
    return dir_list

with open('pad_word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('pad_index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)
np.save('pad_eng.npy', pad_eng)
np.save('pad_cns.npy', pad_cns)
    
print(pad_cns[:3])
print(pad_cns_b[:3])

Using TensorFlow backend.


[[17750  7301  7302 17751     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [17750  7303  7302 17751     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [17750  7304  7305  7306  7307  7302 17751     0     0     0     0     0
      0     0     0     0     0     0     0     0]]
[list([7301, 7302, 17751]) list([7303, 7302, 17751])
 list([7304, 7305, 7306, 7307, 7302, 17751])]
